In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import scipy.stats as stats
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

import os
import glob
import sys

In [2]:
from utils import set_basepath

In [3]:
sys.path[0]

'/Users/jacobhow/Documents/programming-projects/nfl-big-data/notebooks/feature'

In [4]:
basepath = set_basepath(steps=2)

In [5]:
sys.path[0]

'/Users/jacobhow/Documents/programming-projects/nfl-big-data'

In [6]:
from src.team import Team
from src.game import Game
from src.play import Play
from src.player import Player
from src.analysis import process_games, aggregate_coverages

ImportError: cannot import name 'process_games' from 'src.analysis' (/Users/jacobhow/Documents/programming-projects/nfl-big-data/src/analysis.py)

In [ ]:
games = pd.read_csv(os.path.join(basepath,'data/games.csv'))

In [ ]:
players = pd.read_csv(os.path.join(basepath,'data/players.csv')).set_index('nflId')

In [ ]:
plays = pd.read_csv(os.path.join(basepath,'data/plays.csv'))

In [ ]:
weeks = {}
for i in range(17):
    name = f'week{i+1}'
    weeks[name] = pd.read_csv(os.path.join(basepath,'data',name+'-processed.csv'))

In [ ]:
teams = process_games(games,plays)

In [ ]:
team = teams['SEA']

In [ ]:
team.games

In [ ]:
team.process_weeks(weeks)

In [ ]:
for game in team.games.values():
    game.process_plays(players)

In [ ]:
game_coverages = []

for game in team.games.values():
    _coverage = game.classify_defensive_back_coverages(positions='CB',useId=True)
    game_coverages.append(_coverage)

In [ ]:
season_coverages = aggregate_coverages(coverages=game_coverages)

In [ ]:
season_coverages

In [ ]:
df = pd.DataFrame.from_dict(season_coverages, orient='index')

In [ ]:
df

In [ ]:
players.head()

In [ ]:
df = pd.merge(players[['displayName','position']],df,left_index=True,right_index=True)

In [ ]:
df

coverage_names = ('zone','zone-deep','zone-over','man','man-over','blitz')

for key in coverage_names:
    df[key] /= df['snaps']

In [ ]:
df

In [ ]:
SNAP_THRESHOLD = 50

In [ ]:
df = df[df['snaps'] > SNAP_THRESHOLD]

In [ ]:
df['Team'] = team.abbr

In [ ]:
df

In [ ]:
df.index

In [ ]:
s = df.loc[df.index[3]]
s

In [ ]:
coverage_names = ('zone','zone-deep','zone-over','man','man-over','blitz')

In [ ]:
values = []
for key in coverage_names:
    val = s[key]
    values.append(val)

In [ ]:
display_names = []
for name in coverage_names:
    _name=name.title()
    display_names.append(_name)

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))

ax.pie(values,labels=display_names, autopct='%1.1f%%',textprops={'fontsize': 14})
ax.axis('equal') 

player_name = s['displayName']
player_pos = s['position']
title = f'{player_name} - {player_pos}'
ax.set_title(title,fontsize=18)

nGames = s['games played']
nSnaps = s['snaps']

ax.set_xlabel(f'Games Played: {nGames}  -  Total Snaps: {nSnaps}',fontsize=14)

plt.savefig('Coverage Tendencies Example.pn')

plt.show()